In [ ]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
from pathlib import Path
path = Path.cwd()
data_path = path.parent
data_path = str(data_path) + '/data/'
data_path

# 날씨 데이터 불러오기

In [ ]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv(data_path +'Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether

# 차량 데이터 불러오기 및 전처리

# 다래데이터만 > 다래 데이터 형식으로 테이블 맞춰서 제공한다고 하였음

In [ ]:
import pandas as pd

In [ ]:
darae_in = pd.read_csv(data_path+ 'VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv(data_path+'VW_OutCarinfo (차량 출차 정보 조회).csv')

In [ ]:
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

In [ ]:
darae_in['장비명'].value_counts()

In [ ]:
darae_out

## 이름 바꾸기 13_1동 지하 1 > 13

In [ ]:
import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0
for i in range(len(darae_out)):
    slicing(darae_out,i)

def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x
darae_in['주차장'] = darae_in['주차장'].apply(change)
darae_out['주차장'] = darae_out['주차장'].apply(change)

In [ ]:
pd.options.display.float_format = '{:.0f}'.format

In [ ]:
import copy

In [ ]:
test_in = copy.deepcopy(darae_in)
test_in.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_in['입출차여부'] = '입차'

In [ ]:
test_out = copy.deepcopy(darae_out)
test_out.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_out['입출차여부'] = '출차'
test_out[test_out['장비명'].str.contains('입구')]['입출차카드번호'].value_counts()

In [ ]:
df = pd.concat([test_in, test_out]).sort_values(by=['차량번호','일자','시각'])#[['차량번호','입출차여부','일자','시각','입출차구분','출차여부','장비명']]

In [ ]:
import datetime
df['입출차시각'] = df['일자'] + df['시각']
df['입출차시각'] = pd.to_datetime(df['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')
df['시'] = df['입출차시각'].dt.hour
df.reset_index(inplace = True)

In [ ]:
df

# 입차시간 == 출차시간인 경우 파악하기
- 입차시간 == 출차시간인 경우 입차로간주하고 출차 제거하기

In [ ]:
car_arr = np.array(df[['일자','시각','차량번호']])
car_arr

In [ ]:
test_idx_list =[]
for idx in range(len(car_arr)-1):
    if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == car_arr[idx+1][1] and car_arr[idx][2] == car_arr[idx+1][2]:
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
        #test_idx_list.append(idx+2)
test_idx_list = list(set(test_idx_list))
test_df = df.loc[test_idx_list]
eleminate_index = test_df[test_df['입출차여부']=='출차'].index
df2 = df.drop(eleminate_index)
df2.drop(['index'],axis = 1 , inplace = True)
df2.reset_index(inplace=True)
df2.drop(['index'],axis = 1 , inplace = True)
df2

In [ ]:
test_df[test_df['차량번호'] == '85부3062']


# 데이터 전처리
- 아침시간이니 입차시간 == 출차시간인 경우 입차로 가정
- 차량 번호별로 입차1,입차2,.....출차,출차 인경우  >>  입차1, 출차로
- 입차1,입차2,입차3, 출차 >>> 입차3, 출차 로해야하는게 맞는듯?
- 아니면 출차 주차장 기준으로 하든가

In [ ]:
car_arr = np.array(df2[['차량번호','입출차여부','시각']])
idx_list = []
for idx in range(len(car_arr)):
    if idx == 0 :
        idx_list.append(idx)
    else :  # 차량 번호가 같고 상태가 다른 경우(입차 > 출차 or 출차 > 입차)
        if (car_arr[idx][0] == car_arr[idx-1][0]) and (car_arr[idx][1]!=car_arr[idx-1][1]):
            idx_list.append(idx)

            #차량번호가 바뀌는 경우
        elif car_arr[idx][0] != car_arr[idx-1][0]:
            idx_list.append(idx)



In [ ]:
process_df = df2.loc[idx_list]
process_df[process_df['차량번호'] == '85부3062']

In [ ]:
process_df = df2.loc[idx_list]
process_df['입출차여부'].value_counts()

process_df['연'] = process_df['입출차시각'].dt.year
process_df['월'] = process_df['입출차시각'].dt.month
process_df['일'] = process_df['입출차시각'].dt.day
process_df['시'] = process_df['입출차시각'].dt.hour
process_df['분'] = process_df['입출차시각'].dt.minute
process_df['초'] = process_df['입출차시각'].dt.second
process_df = process_df[(process_df['시']>=7) & (process_df['시']<=9)]
process_df

In [ ]:
test = process_df[process_df['일']==15]
test[test['주차장']=='13동']['입출차여부'].value_counts()

In [ ]:
parking_info = pd.read_excel('주차장면수.xlsx')
parking_info

# 데이터셋 생성

In [ ]:
dataset = pd.merge(process_df,parking_info[['장소','합계']], left_on='주차장', right_on='장소')

In [ ]:
dataset = dataset[['입출차시각','연','월','일','시','분','초','입출차여부','주차장','합계']]

In [ ]:
dataset = dataset.sort_values(by = ['주차장','입출차시각'])
dataset['잔류차량수'] = 0

In [ ]:
dataset

In [ ]:
arr = np.array(dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']])
arr

In [ ]:
for idx in range(len(arr)):
    if idx == 0 :
        if arr[idx][5] == '입차' :
            arr[idx][7] = arr[idx][7] +1
        else :
            arr[idx][7] = arr[idx][7] -1
    else :
        pre_time = list(arr[idx-1][[0,1,2,6]])
        now_time = list(arr[idx][[0,1,2,6]])
        status =  arr[idx][5]
            # 이전 데이터와 연,월,일 주차장이 같은 경우
        if pre_time == now_time :
            #print('>')
            if status == '입차' :
                arr[idx][7] = arr[idx-1][7] +1
            else :
                arr[idx][7] = arr[idx-1][7] -1

            #이전 데이터와 연,월,일 주차장이 다른 경우 > 0부터 시작함
        else :
            if status == '입차':
                arr[idx][7] =1
            else :
                arr[idx][7] = -1


In [ ]:
dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']] = arr
dataset

# 혼잡도 만들기

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
dataset['비율']=dataset['잔류차량수']/dataset['합계']

def confusion(df):
    if df['비율'] <= 0.3 :
        return 0
    elif df['비율'] <= 0.5 :
        return 1
    elif df['비율'] <= 0.8 :
        return 2
    else :
        return 3

dataset['혼잡도']=dataset.apply(confusion,axis=1)
dataset

# 날씨데이터 붙이기

In [ ]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv(data_path +'Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether


whether = whether.reset_index()
whether['연'] = whether['일시'].dt.year
whether['월'] = whether['일시'].dt.month
whether['일'] = whether['일시'].dt.day
whether['시'] = whether['일시'].dt.hour
whether

In [ ]:
dataset = pd.merge(dataset,whether, on =['연','월','일','시'], how= 'left')
dataset

In [ ]:
import holidays
kr_holidays = holidays.KR()
def holiday_check(x):
    if x in kr_holidays:
        return 1
    else :
        return 0


dataset['공휴일'] = dataset['입출차시각'].apply(holiday_check)
일
dataset

In [ ]:
dataset = dataset[['입출차시각', '연', '월', '일', '시', '분','초','공휴일','요일','주차장',
         '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)',
         '잔류차량수', '비율','혼잡도']]
# 날짜 매칭이 안되는 데이터들 제거
dataset = dataset.dropna()

In [ ]:
#dataset.to_csv('dataset_darae.csv')

In [214]:
dataset

,입출차시각,연,월,일,시,분,초,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),잔류차량수,비율,혼잡도
0,2022-09-01 07:07:31,2022,9,1,7,7,31,0,3,12동,20.20000,0.00000,0.90000,94.00000,0.00000,1,0.01220,0
1,2022-09-01 07:17:42,2022,9,1,7,17,42,0,3,12동,20.20000,0.00000,0.90000,94.00000,0.00000,2,0.02439,0
2,2022-09-01 07:18:36,2022,9,1,7,18,36,0,3,12동,20.20000,0.00000,0.90000,94.00000,0.00000,3,0.03659,0
3,2022-09-01 07:35:54,2022,9,1,7,35,54,0,3,12동,20.20000,0.00000,0.90000,94.00000,0.00000,4,0.04878,0
4,2022-09-01 07:35:58,2022,9,1,7,35,58,0,3,12동,20.20000,0.00000,0.90000,94.00000,0.00000,3,0.03659,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15518,2022-09-30 09:02:40,2022,9,30,9,2,40,0,4,15동,14.30000,0.00000,0.40000,95.00000,0.00000,72,0.35821,1
15519,2022-09-30 09:02:46,2022,9,30,9,2,46,0,4,15동,14.30000,0.00000,0.40000,95.00000,0.00000,73,0.36318,1
15520,2022-09-30 09:03:58,2022,9,30,9,3,58,0,4,15동,14.30000,0.00000,0.40000,95.00000,0.00000,74,0.36816,1
15521,2022-09-30 09:04:03,2022,9,30,9,4,3,0,4,15동,14.30000,0.00000,0.40000,95.00000,0.00000,75,0.37313,1


# 보정하기
 - 잔류차량수 >주차면수 인 경우 어떻게??
-  생각해봐야함

In [ ]:
a = dataset.groupby(['연','월','일','주차장'])['잔류차량수','합계'].max()
a=pd.DataFrame(a)
a.reset_index(inplace=True)

In [ ]:
a['전체-잔류'] = a['합계'] - a['잔류차량수']
a